In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import os
import os.path as osp
import collections
import PIL
import imageio
from distutils.version import LooseVersion
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# # Downloading dataset from google drive, 밑의 코드의 주석을 풀면 구글 드라이브로 부터 데이터셋 다운, 압축이 풀리고, Kitti라는 폴더가 생성됩니다. 

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19EiycfOQtf6uDKvMgwlHZB50cAxX_U4z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19EiycfOQtf6uDKvMgwlHZB50cAxX_U4z" -O Kitti.zip && rm -rf /tmp/cookies.txt
# !mkdir Kitti
# !unzip Kitti.zip -d Kitti


# Dataloader

In [ ]:
imgsets_file = osp.join('Kitti', '{}.txt'.format('train'))
for line in open(imgsets_file):
    line = line.strip()
#     import pdb; pdb.set_trace()
    print(line)
    line = line.split()

In [ ]:
class KITTIdataset(torch.utils.data.Dataset):
    class_names = np.array(['background', 'road'])

    def __init__(self, root, transform, split='train'): # root: "./Kitti"
        ## split에 맞는 txt파일 읽어서, 
        ## input image label image경로 리스트로 각각 저장
        self.root = root
        self.transform = transform
        self.split = split
        
        self.images_path = []
        self.ys = []
        
        imgsets_file = osp.join('Kitti', '{}.txt'.format(split)) # train, val
        for line in open(imgsets_file):
            #"\n"
            # line[0]: input image path, # line[1]: label image
            #load image file
            #load label file
            self.images_path.append(img_file)
            self.ys.append(lbl_file)
            
    def __len__(self):
        ## length return
        return len(self.ys)
    
    def __getitem__(self, index):
        ## index에 맞는 image, label image읽어오기
        # load image
        
        # load label
        
        ## black: 0, white: 255 255->1
        ## 0, 1
        
        return self.transform(img), torch.from_numpy(lbl).long()

## Define dataset and dataloader

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])
train_dataset = KITTIdataset(root = './Kitti', split = 'train', transform = transform)
val_dataset = KITTIdataset(root = './Kitti', split = 'val', transform = transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1, shuffle = False)

In [ ]:
# print(train_dataset[7][0].shape) ## [0]: img tensor, [1]: label tensor

# Define the Network
-VGG16

- FCN model

In [ ]:
import torch.nn as nn
import torch

# Maxpooling with ceil mode


In [ ]:
#Transposed convolution

In [ ]:
class FCN(nn.Module):
    def __init__(self, num_class = 3):
        super(FCN, self).__init__()
        
        ## conv1
        
        
        ## conv2
        
        ## self.pool2
        
        ## conv3
        
        ### pool3
        ## conv4
        
        ### pool4
        
        ## conv5
        
        ## pool 5
        
        ## upsampling transposed convolution
        
        self.params = [self.features1, self.features2, self.features3, 
                       self.features4, self.features5]
        

                             
    def forward(self, inputs):
        # input [Batch size, 3, w, h]

        
        ## channel num_class
        
        
        return x ## label map [batch, num_class, inputs h, input w]
    
    def copy_params(self, vgg):
        

In [ ]:
# model = FCN(2)
# # # print(model)
# temp_input = torch.rand(1, 3, 1024, 1024)
# output = model(temp_input)
# print(output.shape)

## Measure accuracy and visualization

In [ ]:
def _fast_hist(label_true, label_pred, n_class):
    ## define mask, histogram
    ## hint : np.bincount
    
    return hist

def compute_mean_iou(label_trues, label_preds, n_class): 
    # label_true: [h, w] # label_pred: [h, w] # n_class: 2
    # TO DO : 각 Class 별 Intersection of Union을 계산
        # Hint1 : _fast_hist를 통해 confusion matrix를 구할 수 있다. 
        # Hint2 : 해당 class의 True = confusion matrix의 해당 row 값의 합
        #         해당 class의 Positive = confusion matrix의 해당 column 값의 합 
        #         해당 class의 True_Positive = confusion matrix의 (class, class)의 값
        # Hint3 : iou = True_Positive / True + Positive - True_Positive 

    return mean_iou

def visualization(net, input_img, epoch):
    ## TO DO : image를 network에 넣어 label을 추출
    
    os.makedirs("./pred", exist_ok = True)
    os.makedirs("./input", exist_ok = True)
    
    imageio.imsave('./pred/mask_'+str(epoch+1)+'.png', lbl_pred)
    plt.imshow(mpimg.imread('./pred/mask_'+str(epoch+1)+'.png'),cmap='gray') ### visualize predicted label map
    plt.show()
    input_img = np.array(input_img)
    imageio.imsave('./input/input_'+str(epoch+1)+'.png', input_img)
    plt.imshow(mpimg.imread('./input/input_'+str(epoch+1)+'.png'))
    plt.show()
    

In [ ]:
label_true = np.array([
    [1, 2],
    [2, 3]
])
label_pred = np.array([
    [1, 2],
    [2, 2]
])

compute_mean_iou(label_true, label_pred, 4)

# Train

In [ ]:
##load pretrained model from torchvision
##pretrained using coco 2017
net = torchvision.models.segmentation.fcn_resnet50(pretrained = True)
# print(net)
net.classifier = torchvision.models.segmentation.fcn.FCNHead( 2048, 2)
net = net.cuda()
print(net)

In [ ]:
# vgg16 = torchvision.models.vgg16(pretrained = True)
# net = FCN(num_class = 2)
# net.copy_params(vgg16)

# net = net.cuda()

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=1e-5, weight_decay = 0.0001)

In [ ]:
training_epochs = 5
best_iou = 0
num_class = len(train_loader.dataset.class_names)
j=0

for epoch in range(training_epochs):
    net.train()
    print ('current epoch : %d'%(epoch))
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        # load data, forward
        
        
        if batch_idx % 20 ==0:
            print ('batch : {}, loss : {}'.format(batch_idx, loss.item()))
        j += 1
        
    #validation
    net.eval()
    val_loss = 0
    metrics = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            # load data, forward
            
            # calc val loss, accuracy
            loss = criterion(score, target)

            val_loss += loss.item()

           
    val_loss /= len(val_loader)
    metrics = np.mean(metrics)
    
    print ('val loss : {}, mean_iou : {}'.format(val_loss, metrics))

    ##save model
    if best_iou < metrics:
        best_iou = metrics
        print("Best model saved")
        torch.save(net.state_dict(), './model_best.pth')
    
    ## visualization
    img = PIL.Image.open('./road_sample1.png')
    visualization(net, img, epoch)
print('Finished Training')

In [ ]:
img = PIL.Image.open('./road_sample1.png')
visualization(net, img, epoch)